In [3]:
import pandas as pd
import numpy as np
from typing import Dict, Tuple
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.svm import SVC, NuSVC, LinearSVC, OneClassSVM, SVR, NuSVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
import nltk
import re
from gensim.models import word2vec
from sklearn.naive_bayes import MultinomialNB

In [2]:
test_data = pd.read_csv('SMS_test.csv', encoding= 'unicode_escape')
train_data = pd.read_csv('SMS_train.csv', encoding= 'unicode_escape')

In [6]:
X_train = train_data['Message_body']
X_test = test_data['Message_body']
y_train = train_data['Label']
y_test = test_data['Label']

In [7]:
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res


In [8]:
def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

In [9]:
def sentiment(v, c):
    model = Pipeline(
        [("vectorizer", v), 
         ("classifier", c)])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print_accuracy_score_for_classes(y_test, y_pred)

In [10]:
sentiment(CountVectorizer(), LogisticRegression(C=5.0))

Метка 	 Accuracy
Non-Spam 	 1.0
Spam 	 0.75


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [12]:
sentiment(CountVectorizer(), MultinomialNB())

Метка 	 Accuracy
Non-Spam 	 0.9591836734693877
Spam 	 0.8421052631578947


In [11]:
sentiment(TfidfVectorizer(), LogisticRegression(C=5.0))

Метка 	 Accuracy
Non-Spam 	 1.0
Spam 	 0.6710526315789473


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [13]:
sentiment(TfidfVectorizer(), MultinomialNB())

Метка 	 Accuracy
Non-Spam 	 1.0
Spam 	 0.42105263157894735


Лучшую точность показывает модель с вариантом векторизации признаков CountVectorizer и классификатором MultinomialNB